In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import keras
import tensorflow as tf
from tensorflow import keras
import numpy as np
keras.__version__

In [2]:
import os ,shutil
print (os.getcwd())
path='../input/melanoma-and-benign-images-only/DermMel'
orginal_dataset_dir=path
print(path)

In [3]:
tf.config.experimental.list_physical_devices()


In [4]:
tf.__version__

In [5]:
tf.test.is_built_with_cuda()


In [6]:
base_dir=os.path.join(os.getcwd(),'../input/melanoma-and-benign-images-only/DermMel')
print(base_dir)

if(not(os.path.exists(base_dir))):
    os.mkdir(base_dir)
else:
    print('prepared alraedy exists')
    
    
train_dir=os.path.join(base_dir,'train')   
if(not(os.path.exists(base_dir))):
    os.mkdir(train_dir)
else:
    print('Train_Dir alraedy exists')
    
test_dir=os.path.join(base_dir,'test')
if(not(os.path.exists(test_dir,))):   
    os.mkdir(test_dir)
else:
    print('Test_Dir alraedy exists')
validation_dir=os.path.join(base_dir,'validation')
if(not(os.path.exists(validation_dir))):
    os.mkdir(validation_dir)
else:
    print('validation dir already exists')

In [7]:
train_classes=os.listdir(train_dir)
print("train_class:",train_classes)
test_classes=os.listdir(test_dir)
print("test_classes:",test_classes)
validation_classes=os.listdir(validation_dir)
print('validation :',validation_classes)

In [9]:
from tensorflow.keras.optimizers import Adam
# opt = keras.optimizers.Adam(learning_rate=0.001)
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='binary_crossentropy',optimizer='adam', 
              metrics=['acc'])

In [13]:
print(path)

dst_dataset_dir_classes=os.listdir('../input/melanoma-and-benign-images-only/DermMel')
print(path)
dst_dataset_train_dir=os.path.join(path,'train')
print(dst_dataset_train_dir)

dst_dataset_test_dir=os.path.join(path,'test')
print(dst_dataset_test_dir)

dst_dataset_validation_dir=os.path.join(path,'validation')
print(dst_dataset_validation_dir)



list_train_classes=os.listdir('../input/melanoma-and-benign-images-only/DermMel/train')
print("dst train_class:",list_train_classes)

list_test_classes=os.listdir('../input/melanoma-and-benign-images-only/DermMel/test')
print("dst test_classes:",list_test_classes)

list_validation_classes=os.listdir('../input/melanoma-and-benign-images-only/DermMel/validation')
print('dst validation :',list_validation_classes)



In [79]:
from keras import layers
from keras import models
from keras import optimizers

# def get_model():
#     model = models.Sequential([
#     model.add(layers.Conv2D(256, (3, 3), activation='relu',input_shape(224,224,3)))
#     model.add(layers.Dense(3000, activation='relu'))
#     model.add(layers.Dense(1000, activation='relu'))
#     model.add(layers.Dense(1, activation='sigmoid'))]
        
def get_model():
    model = keras.Sequential([
            keras.layers.Flatten(input_shape=(224,224,3)),
            keras.layers.Dense(3000, activation='relu'),
            keras.layers.Dense(1000, activation='relu'),
            keras.layers.Dense(1, activation='sigmoid')]) 
    
    model.compile(optimizer='SGD',loss='binary_crossentropy',metrics=['accuracy'])
    return model
        
    model.summary()

In [80]:
train_tf_dataset = tf.data.Dataset.from_tensor_slices((list_train_classes))
test_tf_dataset = tf.data.Dataset.from_tensor_slices((list_test_classes))

In [81]:
strategy = tf.distribute.MirroredStrategy()


In [82]:
strategy.num_replicas_in_sync


In [83]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen=ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(
        dst_dataset_train_dir,
            target_size=(224,224),
            batch_size=20,
        class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
        dst_dataset_validation_dir,
        target_size=(224, 224),
        batch_size=20,
        class_mode='binary')

testing_generator = test_datagen.flow_from_directory(
        dst_dataset_test_dir,
        target_size=(224, 224),
        batch_size=20,
        class_mode='binary')

In [84]:
BATCH_SIZE_PER_REPLICA = 250
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync


train_dataset = train_tf_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_tf_dataset.batch(BATCH_SIZE)

In [85]:
from PIL import ImageFile
# ImageFile.LOAD_TRUNCATED_IMAGES = True

# model.compile('adam','binary_crossentropy',metrics=['acc'])
# history = model.fit_generator(
#       train_generator,
#       steps_per_epoch=100,
#       epochs=50,
#       validation_data=validation_generator,
#       validation_steps=100)
with strategy.scope():
    gpu_model = get_model()
    history=gpu_model.fit_generator(train_generator,
                  steps_per_epoch=100,
                  epochs=50,
                  validation_data=validation_generator,
                 validation_steps=100)


In [91]:
import matplotlib.pyplot as plt

acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history["loss"]
val_loss=history.history['val_loss']
epochs=range(len(acc))
plt.plot(epochs,acc,'bo',label='Training acc')
plt.plot(epochs,val_acc,label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()        
plt.figure()
plt.plot(epochs,loss,'bo',label='Training loss')
plt.plot(epochs,val_loss,label='Validation loss')
plt.title('Training and validation loss')
plt.legend()         
plt.figure()
plt.show()